In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from collections import Counter
import scipy.sparse as sparse
from tensorrec import TensorRec
import os
filepath = 'data/'
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df_repurchase = pd.read_csv(filepath + 'NRA_Repurchase_ANP_Jan_Apr_2021.csv')
df_repurchase.shape

(155753, 3)

In [3]:
df = pd.read_csv(filepath + 'NRA_VALIDATION_DEC_2020.csv')
df.shape

(2438540, 52)

In [4]:
df.drop_duplicates(subset='INS_ID',keep='first',inplace=True, ignore_index=True)
df.shape

(2403078, 52)

In [5]:
df1 = df[['INS_ID', 
       'REF_DT', 'INS_AGE', 'INS_GENDER', 'INS_LIFE_STAGE', 'INS_NON_ACUTE',
       'INS_CLM_MAJOR', 'INS_ACUTE_CT_6', 'INS_PAID_AMT_ACUTE_6',
       'INS_ACCIDENT_CT_6', 'INS_PAID_AMT_ACCIDENT_6', 'INS_INT_CALL_3M',
       'INS_KID_CT', 'INS_CITY', 'INS_SMOKER_IND', 'INS_LAPSE_12M',
       'INS_SUR_POL_12M', 'INS_PP_POL', 'INS_ETI_RPU', 'INS_POL_APL_PH',
       'INS_POL_LOAN_IND', 'INS_LATE_PAYMENT_3M', 'INS_LATE_PAYMENT_12M',
       'INS_RECENCY', 'INS_BILL_AMT', 'INS_BAD_RISK', 'INS_BAD_RISK_CT_24',
       'INS_PAID_AMT_BAD_RISK', 'INS_PAID_AMT_BAD_RISK_24', 'INS_CS_BAD_RISK',
       'INS_CS_BAD_RISK_CT_24', 'VIT_ACT_3M', 'INS_INT_EMAIL_3M',
       'INS_INT_MYAIA_TRAN_3M', 'INS_INCOME_SEGM']]
df1.head()

INS_ID      REF_DT  INS_AGE INS_GENDER INS_LIFE_STAGE  INS_NON_ACUTE  \
0   1114009  2020-12-31     29.0          M   Young Couple              0   
1  16809666  2020-12-31      2.0          F        Minors               1   
2   8963869  2020-12-31     20.0          F   Young Single              0   
3   2139723  2020-12-31     58.0          F    Golden Ager              0   
4   5880459  2020-12-31     49.0          F  Mature Couple              0   

   INS_CLM_MAJOR  INS_ACUTE_CT_6  INS_PAID_AMT_ACUTE_6  INS_ACCIDENT_CT_6  \
0              0               0                   0.0                  0   
1              0               0                   0.0                  0   
2              0               0                   0.0                  0   
3              0               0                   0.0                  0   
4              0               0                   0.0                  0   

   ...  INS_BAD_RISK  INS_BAD_RISK_CT_24  INS_PAID_AMT_BAD_RISK  \
0  ...             0                   0                    0.0   
1  ...             0                   0                    0.0   
2  ...             0                   0                    0.0   
3  ...             0                   0                    0.0   
4  ...             0                   0                    0.0   

  INS_PAID_AMT_BAD_RISK_24  INS_CS_BAD_RISK  INS_CS_BAD_RISK_CT_24  \
0                      0.0                0                      0   
1                      0.0                0                      0   
2                      0.0                0                      0   
3                      0.0                0                      0   
4                      0.0                0                      0   

   VIT_ACT_3M  INS_INT_EMAIL_3M  INS_INT_MYAIA_TRAN_3M    INS_INCOME_SEGM  
0           0                 0                      0      G1 Low Income  
1           0                 0                      0                 G0  
2           0                 0                      0                 G0  
3           0                 0                      0                 G0  
4           0                 0                      0  G12 Mass Affluent  

[5 rows x 35 columns]

In [6]:
df1.isnull().sum().sort_values(ascending=False)

INS_AGE                     96
INS_LIFE_STAGE              96
INS_ID                       0
INS_PAID_AMT_BAD_RISK        0
INS_LATE_PAYMENT_3M          0
INS_LATE_PAYMENT_12M         0
INS_RECENCY                  0
INS_BILL_AMT                 0
INS_BAD_RISK                 0
INS_BAD_RISK_CT_24           0
INS_PAID_AMT_BAD_RISK_24     0
INS_POL_APL_PH               0
INS_CS_BAD_RISK              0
INS_CS_BAD_RISK_CT_24        0
VIT_ACT_3M                   0
INS_INT_EMAIL_3M             0
INS_INT_MYAIA_TRAN_3M        0
INS_POL_LOAN_IND             0
INS_PP_POL                   0
INS_ETI_RPU                  0
REF_DT                       0
INS_SUR_POL_12M              0
INS_LAPSE_12M                0
INS_SMOKER_IND               0
INS_CITY                     0
INS_KID_CT                   0
INS_INT_CALL_3M              0
INS_PAID_AMT_ACCIDENT_6      0
INS_ACCIDENT_CT_6            0
INS_PAID_AMT_ACUTE_6         0
INS_ACUTE_CT_6               0
INS_CLM_MAJOR                0
INS_NON_

In [7]:
df1.dropna(inplace =True)
df1.shape

C:\Users\I008328\.conda\envs\kw-py37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(2402982, 35)

In [8]:
df1.isnull().sum().sort_values(ascending=False).head()

INS_ID                  0
INS_BAD_RISK_CT_24      0
INS_POL_LOAN_IND        0
INS_LATE_PAYMENT_3M     0
INS_LATE_PAYMENT_12M    0
dtype: int64

In [9]:
df2 = df1.copy()

In [10]:
df_train = pd.get_dummies(data=df2,columns=['INS_LIFE_STAGE','INS_INCOME_SEGM'])

In [11]:
df_train.columns

Index(['INS_ID', 'REF_DT', 'INS_AGE', 'INS_GENDER', 'INS_NON_ACUTE',
       'INS_CLM_MAJOR', 'INS_ACUTE_CT_6', 'INS_PAID_AMT_ACUTE_6',
       'INS_ACCIDENT_CT_6', 'INS_PAID_AMT_ACCIDENT_6', 'INS_INT_CALL_3M',
       'INS_KID_CT', 'INS_CITY', 'INS_SMOKER_IND', 'INS_LAPSE_12M',
       'INS_SUR_POL_12M', 'INS_PP_POL', 'INS_ETI_RPU', 'INS_POL_APL_PH',
       'INS_POL_LOAN_IND', 'INS_LATE_PAYMENT_3M', 'INS_LATE_PAYMENT_12M',
       'INS_RECENCY', 'INS_BILL_AMT', 'INS_BAD_RISK', 'INS_BAD_RISK_CT_24',
       'INS_PAID_AMT_BAD_RISK', 'INS_PAID_AMT_BAD_RISK_24', 'INS_CS_BAD_RISK',
       'INS_CS_BAD_RISK_CT_24', 'VIT_ACT_3M', 'INS_INT_EMAIL_3M',
       'INS_INT_MYAIA_TRAN_3M', 'INS_LIFE_STAGE_Established Family',
       'INS_LIFE_STAGE_Golden Ager', 'INS_LIFE_STAGE_Mature Couple',
       'INS_LIFE_STAGE_Mature Single', 'INS_LIFE_STAGE_Minors ',
       'INS_LIFE_STAGE_Nest Builder', 'INS_LIFE_STAGE_Young Couple',
       'INS_LIFE_STAGE_Young Single', 'INS_INCOME_SEGM_G0',
       'INS_INCOME_SEGM

In [12]:
df_train.isnull().sum().sort_values(ascending=False).head()

INS_ID                   0
REF_DT                   0
VIT_ACT_3M               0
INS_INT_EMAIL_3M         0
INS_INT_MYAIA_TRAN_3M    0
dtype: int64

In [13]:
df_train1 = df_train[['INS_RECENCY','INS_AGE','INS_BILL_AMT','INS_PP_POL','INS_KID_CT','INS_INT_MYAIA_TRAN_3M','INS_NON_ACUTE','INS_PAID_AMT_ACUTE_6',
'INS_SMOKER_IND','INS_ACUTE_CT_6','VIT_ACT_3M','INS_INT_CALL_3M','INS_LATE_PAYMENT_12M','INS_POL_APL_PH','INS_INCOME_SEGM_G6 Mass Affluent',
'INS_INCOME_SEGM_G3 Mass','INS_LIFE_STAGE_Young Single','INS_LIFE_STAGE_Mature Single', 'INS_LIFE_STAGE_Nest Builder','INS_INCOME_SEGM_G14 ENHW/HNW',
'INS_CLM_MAJOR','INS_PAID_AMT_ACCIDENT_6','INS_LIFE_STAGE_Mature Couple','INS_INCOME_SEGM_G2 Mass','INS_INCOME_SEGM_G7 Mass Affluent',
 'INS_INCOME_SEGM_G11 Mass Affluent','INS_LIFE_STAGE_Established Family','INS_ACCIDENT_CT_6','INS_INCOME_SEGM_G13 Mass Affluent',
'INS_LIFE_STAGE_Young Couple','INS_LATE_PAYMENT_3M','INS_INCOME_SEGM_G5 Mass Affluent','INS_LIFE_STAGE_Minors ',
'INS_INCOME_SEGM_G9 Mass Affluent','INS_PAID_AMT_BAD_RISK','INS_LIFE_STAGE_Golden Ager','INS_INCOME_SEGM_G4 Mass','INS_CS_BAD_RISK',
'INS_INT_EMAIL_3M','INS_INCOME_SEGM_G8 Mass Affluent','INS_BAD_RISK','INS_INCOME_SEGM_G12 Mass Affluent',
 'INS_INCOME_SEGM_G10 Mass Affluent','INS_PAID_AMT_BAD_RISK_24']]

In [14]:
user_f  = sparse.coo_matrix(df_train1) 
item_f  = sparse.identity(11)

In [16]:
# load model for prediction
model_pred = TensorRec.load_model('saved_models/rec_age_married_wo_inverse')

INFO:tensorflow:Restoring parameters from saved_models/rec_age_married_wo_inverse\tensorrec_session.cpkt


In [17]:
predicted_ranks = model_pred.predict_rank(user_features=user_f,
                                     item_features=item_f)
predicted_ranks

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.


array([[ 2,  5,  1, ...,  4,  9,  3],
       [ 2,  8,  1, ...,  5, 10,  4],
       [ 2,  7,  1, ...,  5,  9,  4],
       ...,
       [ 2,  6,  1, ...,  5,  9,  3],
       [ 2,  7,  1, ...,  5,  9,  3],
       [ 2,  5,  1, ...,  4,  9,  3]])

In [18]:
cols = ['FLG_21','FLG_MARRIED', 'FLG_LIFE_STAGE', 'FLG_INCOME_CHANGE','FLG_POL_ANNIVERSARY_1M','FLG_BIRTHDAY',
                                 'FLG_CLM_TRIGGER','FLG_NON_ACUTE_CLM_FAM', 'FLG_ACUTE_CLM_FAM','FLG_ACC_CLM_FAM',
                                 'FLG_CALL_TRIGGER']

In [19]:
ranks_df = pd.DataFrame(predicted_ranks)
ranks_df.columns = cols # item_category_scaled.index
ranks_df.index = df1['INS_ID'] #customer_features.index[:-1000]
ranks_df = ranks_df.T

In [20]:
# Predicted dataframe
ranks_df

INS_ID                  1114009   16809666  8963869   2139723   5880459   \
FLG_21                         2         2         2         2         2   
FLG_MARRIED                    5         8         7         6         6   
FLG_LIFE_STAGE                 1         1         1         1         1   
FLG_INCOME_CHANGE              6         6         8         7         7   
FLG_POL_ANNIVERSARY_1M        11        11        11        11        11   
FLG_BIRTHDAY                  10         9        10        10        10   
FLG_CLM_TRIGGER                8         3         3         5         5   
FLG_NON_ACUTE_CLM_FAM          7         7         6         8         8   
FLG_ACUTE_CLM_FAM              4         5         5         4         4   
FLG_ACC_CLM_FAM                9        10         9         9         9   
FLG_CALL_TRIGGER               3         4         4         3         3   

INS_ID                  20583930  1468795   16618569  12219330  16125594  ...  \
FLG_21                         5         2         5         2         2  ...   
FLG_MARRIED                    3         5         3         6         6  ...   
FLG_LIFE_STAGE                 1         1         1         1         1  ...   
FLG_INCOME_CHANGE              4         6         4         7         7  ...   
FLG_POL_ANNIVERSARY_1M        10        11        10        11        11  ...   
FLG_BIRTHDAY                  11        10        11        10        10  ...   
FLG_CLM_TRIGGER                9         7         9         5         5  ...   
FLG_NON_ACUTE_CLM_FAM          7         8         7         8         8  ...   
FLG_ACUTE_CLM_FAM              6         4         6         4         4  ...   
FLG_ACC_CLM_FAM                8         9         8         9         9  ...   
FLG_CALL_TRIGGER               2         3         2         3         3  ...   

INS_ID                  14770672  9040688   241636    1667700   1963323   \
FLG_21                         2         2         2         3         2   
FLG_MARRIED                    5         6         5         4         6   
FLG_LIFE_STAGE                 1         1         1         1         1   
FLG_INCOME_CHANGE              6         8         6         6         8   
FLG_POL_ANNIVERSARY_1M        11        11        11        10        11   
FLG_BIRTHDAY                  10        10        10        11        10   
FLG_CLM_TRIGGER                7         4         7         8         5   
FLG_NON_ACUTE_CLM_FAM          8         7         8         7         7   
FLG_ACUTE_CLM_FAM              4         5         4         5         4   
FLG_ACC_CLM_FAM                9         9         9         9         9   
FLG_CALL_TRIGGER               3         3         3         2         3   

INS_ID                  10157384  1870910   10711642  5125823   442742    
FLG_21                         2         2         2         2         2  
FLG_MARRIED                    7         4         6         7         5  
FLG_LIFE_STAGE                 1         1         1         1         1  
FLG_INCOME_CHANGE              8         6         8         8         6  
FLG_POL_ANNIVERSARY_1M        11        11        11        11        11  
FLG_BIRTHDAY                  10        10        10        10        10  
FLG_CLM_TRIGGER                4         8         4         4         8  
FLG_NON_ACUTE_CLM_FAM          6         7         7         6         7  
FLG_ACUTE_CLM_FAM              5         5         5         5         4  
FLG_ACC_CLM_FAM                9         9         9         9         9  
FLG_CALL_TRIGGER               3         3         3         3         3  

[11 rows x 2402982 columns]

In [21]:
df5 = ranks_df.T
df5.head()

FLG_21  FLG_MARRIED  FLG_LIFE_STAGE  FLG_INCOME_CHANGE  \
INS_ID                                                             
1114009        2            5               1                  6   
16809666       2            8               1                  6   
8963869        2            7               1                  8   
2139723        2            6               1                  7   
5880459        2            6               1                  7   

          FLG_POL_ANNIVERSARY_1M  FLG_BIRTHDAY  FLG_CLM_TRIGGER  \
INS_ID                                                            
1114009                       11            10                8   
16809666                      11             9                3   
8963869                       11            10                3   
2139723                       11            10                5   
5880459                       11            10                5   

          FLG_NON_ACUTE_CLM_FAM  FLG_ACUTE_CLM_FAM  FLG_ACC_CLM_FAM  \
INS_ID                                                                
1114009                       7                  4                9   
16809666                      7                  5               10   
8963869                       6                  5                9   
2139723                       8                  4                9   
5880459                       8                  4                9   

          FLG_CALL_TRIGGER  
INS_ID                      
1114009                  3  
16809666                 4  
8963869                  4  
2139723                  3  
5880459                  3

In [22]:
df6 = df5[['FLG_MARRIED','FLG_INCOME_CHANGE', 'FLG_CALL_TRIGGER','FLG_LIFE_STAGE', 'FLG_POL_ANNIVERSARY_1M','FLG_BIRTHDAY',
                                 'FLG_CLM_TRIGGER','FLG_NON_ACUTE_CLM_FAM', 'FLG_ACUTE_CLM_FAM','FLG_ACC_CLM_FAM',
                                 'FLG_21',]].apply(pd.Series.value_counts)
df6

FLG_MARRIED  FLG_INCOME_CHANGE  FLG_CALL_TRIGGER  FLG_LIFE_STAGE  \
1           NaN                NaN             22473         2378558   
2        3038.0              383.0            420096            4990   
3      211542.0           509032.0           1297313            2458   
4      733527.0           184055.0            153667             526   
5      422601.0           207150.0            501361             536   
6      686326.0           457765.0              6595             420   
7      316953.0           434952.0               422             453   
8       25900.0           603201.0               175            1102   
9         910.0             6177.0                98            1156   
10       1457.0              265.0               146           12531   
11        728.0                2.0               636             252   

    FLG_POL_ANNIVERSARY_1M  FLG_BIRTHDAY  FLG_CLM_TRIGGER  \
1                        1           NaN              387   
2                     1331           1.0           511559   
3                      248          64.0           152973   
4                     8037        1017.0           315497   
5                     1864         516.0           537192   
6                      991        3902.0            44210   
7                      905        2894.0            94136   
8                     2637        2462.0           387900   
9                    24909       62380.0           320469   
10                  420464     1908925.0            12764   
11                 1941595      420821.0            25895   

    FLG_NON_ACUTE_CLM_FAM  FLG_ACUTE_CLM_FAM  FLG_ACC_CLM_FAM   FLG_21  
1                     NaN                NaN              NaN     1563  
2                     NaN                1.0           2137.0  1459446  
3                    29.0              110.0          12024.0   217189  
4                  4655.0           958902.0            726.0    42373  
5                 11597.0           540345.0            792.0   179028  
6                771122.0           365265.0           1235.0    65151  
7               1071977.0            31164.0          11173.0   437953  
8                542781.0           474159.0         362467.0      198  
9                   509.0            13605.0        1972758.0       11  
10                  312.0             6516.0          39536.0       66  
11                    NaN            12915.0            134.0        4

In [23]:
df6 = df6.fillna(0)
df6 = df6/1000

In [24]:
plt.figure(figsize = (18,10))

#create heatmap
sns.heatmap(df6,annot=True,cmap="Blues")

<AxesSubplot:>

In [35]:
print('Ranking 1 {}'.format(list(df5.iloc[0,:][df5.iloc[0,:].isin([1])==True].index)))
print('Ranking 2 {}'.format(list(df5.iloc[0,:][df5.iloc[0,:].isin([2])==True].index)))
print('Ranking 3 {}'.format(list(df5.iloc[0,:][df5.iloc[0,:].isin([3])==True].index)))

Ranking 1 ['FLG_LIFE_STAGE']
Ranking 2 ['FLG_21']
Ranking 3 ['FLG_CALL_TRIGGER']


In [25]:
import time
start_time = time.time()
arr1 = df5.values
flags_mask = np.empty((df5.shape[0],3),dtype=object)
for i in range(df5.shape[0]):
    flags = ['FLG_21', 'FLG_MARRIED', 'FLG_LIFE_STAGE', 'FLG_INCOME_CHANGE',
       'FLG_POL_ANNIVERSARY_1M', 'FLG_BIRTHDAY', 'FLG_CLM_TRIGGER',
       'FLG_NON_ACUTE_CLM_FAM', 'FLG_ACUTE_CLM_FAM', 'FLG_ACC_CLM_FAM',
       'FLG_CALL_TRIGGER']
    arr = arr1[i] # df5.iloc[i].values
    arr, flags = zip(*sorted(zip(arr,flags)))
    flags_mask[i,0] = flags[0]
    flags_mask[i,1] = flags[1]
    flags_mask[i,2] = flags[2]
    
elapsed = time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time))
print(f"Took: {elapsed}")

Took: 00:00:17


In [26]:
df_top3 = pd.DataFrame(flags_mask,index=df5.index,columns=['TOP1_FLAG','TOP2_FLAG','TOP3_FLAG'])
df_top3.head()

TOP1_FLAG TOP2_FLAG         TOP3_FLAG
INS_ID                                              
1114009   FLG_LIFE_STAGE    FLG_21  FLG_CALL_TRIGGER
16809666  FLG_LIFE_STAGE    FLG_21   FLG_CLM_TRIGGER
8963869   FLG_LIFE_STAGE    FLG_21   FLG_CLM_TRIGGER
2139723   FLG_LIFE_STAGE    FLG_21  FLG_CALL_TRIGGER
5880459   FLG_LIFE_STAGE    FLG_21  FLG_CALL_TRIGGER

In [27]:
df_top3.reset_index(inplace=True)

In [28]:
df_top3.head()

INS_ID       TOP1_FLAG TOP2_FLAG         TOP3_FLAG
0   1114009  FLG_LIFE_STAGE    FLG_21  FLG_CALL_TRIGGER
1  16809666  FLG_LIFE_STAGE    FLG_21   FLG_CLM_TRIGGER
2   8963869  FLG_LIFE_STAGE    FLG_21   FLG_CLM_TRIGGER
3   2139723  FLG_LIFE_STAGE    FLG_21  FLG_CALL_TRIGGER
4   5880459  FLG_LIFE_STAGE    FLG_21  FLG_CALL_TRIGGER

In [32]:
df_repurchase.shape

(155753, 3)

In [33]:
df_repurchase.SCV_ID.nunique()

155753

In [45]:
132917-96586

36331

In [34]:
df_pred = pd.merge(df_top3,df_repurchase,how='left',left_on='INS_ID',right_on='SCV_ID')
df_pred.head()

INS_ID       TOP1_FLAG TOP2_FLAG         TOP3_FLAG  SCV_ID  \
0   1114009  FLG_LIFE_STAGE    FLG_21  FLG_CALL_TRIGGER     NaN   
1  16809666  FLG_LIFE_STAGE    FLG_21  FLG_CALL_TRIGGER     NaN   
2   8963869  FLG_LIFE_STAGE    FLG_21  FLG_CALL_TRIGGER     NaN   
3   2139723  FLG_LIFE_STAGE    FLG_21  FLG_CALL_TRIGGER     NaN   
4   5880459  FLG_LIFE_STAGE    FLG_21  FLG_CALL_TRIGGER     NaN   

   REPURCHASE_ANP  TARGET_REPUR_IND  
0             NaN               NaN  
1             NaN               NaN  
2             NaN               NaN  
3             NaN               NaN  
4             NaN               NaN

In [35]:
df_pred.shape

(2402982, 7)

In [36]:
df_pred[df_pred.TARGET_REPUR_IND == 1].shape

(52029, 7)

In [37]:
df_pred[df_pred.INS_ID==df_pred.SCV_ID]

INS_ID       TOP1_FLAG         TOP2_FLAG         TOP3_FLAG  \
33        5861545  FLG_LIFE_STAGE  FLG_CALL_TRIGGER       FLG_MARRIED   
75        2175205  FLG_LIFE_STAGE   FLG_CLM_TRIGGER  FLG_CALL_TRIGGER   
208       3113886  FLG_LIFE_STAGE            FLG_21  FLG_CALL_TRIGGER   
259        657131  FLG_LIFE_STAGE            FLG_21  FLG_CALL_TRIGGER   
301      17263435  FLG_LIFE_STAGE            FLG_21  FLG_CALL_TRIGGER   
...           ...             ...               ...               ...   
2402747   6320975  FLG_LIFE_STAGE            FLG_21  FLG_CALL_TRIGGER   
2402770   2690266  FLG_LIFE_STAGE  FLG_CALL_TRIGGER            FLG_21   
2402813    166321  FLG_LIFE_STAGE  FLG_CALL_TRIGGER       FLG_MARRIED   
2402841  10636686  FLG_LIFE_STAGE       FLG_MARRIED  FLG_CALL_TRIGGER   
2402848   9992384  FLG_LIFE_STAGE            FLG_21  FLG_CALL_TRIGGER   

             SCV_ID  REPURCHASE_ANP  TARGET_REPUR_IND  
33        5861545.0          1042.0               1.0  
75        2175205.0           253.0               1.0  
208       3113886.0           216.0               1.0  
259        657131.0          3188.0               1.0  
301      17263435.0           218.0               1.0  
...             ...             ...               ...  
2402747   6320975.0         31700.0               1.0  
2402770   2690266.0          4559.0               1.0  
2402813    166321.0          2875.0               1.0  
2402841  10636686.0          1194.0               1.0  
2402848   9992384.0           798.0               1.0  

[52029 rows x 7 columns]

In [39]:
52029/2402982

0.021651847579382617

In [40]:
df_pred1 = df_pred[df_pred.TARGET_REPUR_IND == 1]
df_pred1.head()

INS_ID       TOP1_FLAG         TOP2_FLAG         TOP3_FLAG      SCV_ID  \
33    5861545  FLG_LIFE_STAGE  FLG_CALL_TRIGGER       FLG_MARRIED   5861545.0   
75    2175205  FLG_LIFE_STAGE   FLG_CLM_TRIGGER  FLG_CALL_TRIGGER   2175205.0   
208   3113886  FLG_LIFE_STAGE            FLG_21  FLG_CALL_TRIGGER   3113886.0   
259    657131  FLG_LIFE_STAGE            FLG_21  FLG_CALL_TRIGGER    657131.0   
301  17263435  FLG_LIFE_STAGE            FLG_21  FLG_CALL_TRIGGER  17263435.0   

     REPURCHASE_ANP  TARGET_REPUR_IND  
33           1042.0               1.0  
75            253.0               1.0  
208           216.0               1.0  
259          3188.0               1.0  
301           218.0               1.0

In [47]:
df_pred1[['TOP1_FLAG','TOP2_FLAG','TOP3_FLAG']].apply(pd.Series.value_counts,normalize=True).fillna(0)

TOP1_FLAG  TOP2_FLAG  TOP3_FLAG
FLG_21                   0.004863   0.452671   0.096735
FLG_ACC_CLM_FAM          0.000000   0.000038   0.000346
FLG_ACUTE_CLM_FAM        0.000000   0.000846   0.000365
FLG_CALL_TRIGGER         0.013108   0.178497   0.766188
FLG_CLM_TRIGGER          0.019028   0.219128   0.015376
FLG_INCOME_CHANGE        0.000000   0.000000   0.038997
FLG_LIFE_STAGE           0.963001   0.029118   0.006477
FLG_MARRIED              0.000000   0.119683   0.075516
FLG_POL_ANNIVERSARY_1M   0.000000   0.000019   0.000000

In [42]:
df_pred1 = df_pred[df_pred.TARGET_REPUR_IND == 1] #'FLG_NON_ACUTE_CLM_FAM', 'FLG_ACUTE_CLM_FAM', 'FLG_ACC_CLM_FAM'
#'FLG_MARRIED', 'FLG_LIFE_STAGE', 'FLG_INCOME_CHANGE'

In [43]:
df_pred1[df_pred1['TOP1_FLAG'].isin(['FLG_MARRIED', 'FLG_LIFE_STAGE', 'FLG_INCOME_CHANGE'])].shape

(50104, 7)

In [44]:
df_pred1[df_pred1['TOP2_FLAG'].isin(['FLG_MARRIED','FLG_CALL_TRIGGER','FLG_INCOME_CHANGE'])].shape

(15514, 7)

In [45]:
df_pred1[df_pred1['TOP3_FLAG'].isin(['FLG_MARRIED','FLG_CALL_TRIGGER','FLG_INCOME_CHANGE'])].shape

(45822, 7)

In [46]:
df_pred1[df_pred1['TOP3_FLAG'].isin(['FLG_MARRIED','FLG_CALL_TRIGGER','FLG_INCOME_CHANGE'])].shape

(45822, 7)